A6000

注意，CPU不要跑Float16版本，会慢死的

In [1]:
!pip install tensorflow-hub sentencepiece jieba --user

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [2]:
import tensorflow as tf
import tensorflow_hub as hub

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print(gpu)
    tf.config.experimental.set_memory_growth(gpu, True)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [4]:
!/usr/bin/nvidia-smi

Thu Aug 26 17:24:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  RTX A6000           On   | 00000000:41:00.0 Off |                  Off |
| 30%   34C    P8    20W / 300W |      8MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!ls | grep PanGu || git clone https://github.com/qhduan/PanGu-alpha-tf.git

PanGu-alpha-tf


In [6]:
import sys
sys.path.append('PanGu-alpha-tf')

from tokenization_jieba import JIEBATokenizer

In [7]:
print(tf.__version__)

2.4.1


In [8]:
print(hub.__version__)

0.12.0


In [9]:
def sample(tokenizer, gpt, sentence, number=1, length=20, top_k=50, top_p=0.9, temperature=0.9):
    """
    numbert: 输出句子个数
    length: 输出最大长度
    top_k: 只选择前k个，如果为0则为无限制
    top_p: token的概率排在这以上才有效
    temperature: 温度，如果为1.0则相当于无效
    """
    inputs = tf.constant([tokenizer.encode(sentence)] * number, dtype=tf.int64)
    length = tf.constant(length, dtype=tf.int64)
    ret = gpt.signatures['serving_default'](
        inp=inputs,
        length=length,
        top_k=tf.constant(top_k, tf.int32),
        top_p=tf.constant(top_p, tf.float16),
        temperature=tf.constant(temperature, tf.float16)
    )['output_0']
    return [
        tokenizer.decode([int(x) for x in s]).replace(' ', '')
        for s in ret.numpy()
    ]

In [10]:
tokenizer = JIEBATokenizer(
    'PanGu-alpha-tf/PanGu-Alpha-GPU/panguAlpha_pytorch/megatron/tokenizer/bpe_4w_pcl/vocab.vocab',
    'PanGu-alpha-tf/PanGu-Alpha-GPU/panguAlpha_pytorch/megatron/tokenizer/bpe_4w_pcl/vocab.model')

In [11]:
gpt = hub.load('./pangu-13B-tf2-fp16/')

In [13]:
!/usr/bin/nvidia-smi

Thu Aug 26 17:27:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  RTX A6000           On   | 00000000:41:00.0 Off |                  Off |
| 30%   36C    P8    21W / 300W |  33231MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [14]:
%%time
q = '''李大嘴：“各回各家，各找各妈！”
白展堂：“'''
ret = sample(tokenizer, gpt, q, 3, 50)
for x in ret:
    print(x)
    print('-' * 20)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.889 seconds.
Prefix dict has been built successfully.


李大嘴:“各回各家,各找各妈!”
白展堂:“你丫的是啥意思?”
李大嘴:“我是说,今天你不找我揍你,你以后就别来找我揍你!”
白展堂:“......”

--------------------
李大嘴:“各回各家,各找各妈!”
白展堂:“我说,要是我找不着呢?”
李大嘴:“就当你还没找着。”
于小彤:“我怎么没找着?”
李大嘴:“我这不是在你
--------------------
李大嘴:“各回各家,各找各妈!”
白展堂:“李大嘴,你想不想听我说相声?”
李大嘴:“想啊,你说吧!”
白展堂:“我是说你想不想听我唱一首东北大鼓?”
李大
--------------------
CPU times: user 38.7 s, sys: 1.52 s, total: 40.3 s
Wall time: 38.1 s


In [15]:
%%time
q = '''关于爱情，李白曾经说过：“'''
ret = sample(tokenizer, gpt, q, 3, 200)
for x in ret:
    print(x)
    print('-' * 20)

关于爱情,李白曾经说过:“夫天地者,万物之逆旅也光阴者,百代之过客也。而浮生若梦,为欢几何?”每个人的人生都是一段旅程,不管你选择了哪条路,在旅途中会遇到许许多多的人,有人会为你驻足,有人会为你驻足为你停留,他们都会陪你走完一段旅程。但是,爱情是什么?爱情是让人感到温暖的东西,但也是让人感到寒冷的东西。
爱情有时候很美好,有时候却又让人感到伤感。很多人都说过,爱情是人类最美好的情感,但是,人们却也总是在现实中发现,在爱情中,最让人难以忍受的却是现实。
在爱情的世界里,我们每个人都渴望能够找到自己的另一半,但是,又有多少人能够在茫茫人海中找到自己的另一半呢?在茫茫人海
--------------------
关于爱情,李白曾经说过:“妾在梁,郎在魏,妾身未动心,郎心已变”。
我也很赞同这句话。
自从在梁,我们就爱上了那个温柔善良的“妾”。
从她对我的爱中,我可以感受到,梁,就是我的家。
她的家,不仅仅是她的小窝,还是我的家。
我的家,不仅仅是我的家,还是她的家。
她,就是我的“妾”。
只是我从来没有想到,我们会成为“夫妻”。
“妾”,本是个温婉的名字。
在我的印象中,“妾”是个温婉的名字。
记得小时候,最喜欢的事情就是和哥哥姐姐一起玩捉迷藏。
就是在这种情况下,一个
--------------------
关于爱情,李白曾经说过:“爱情诚可贵,生命价更高。若为自由故,二者皆可抛。”爱情和自由,这是一个永恒的话题。有人说,爱情是世界上最美好的东西,也有人说,爱情是人类最珍贵的东西。每一个人都希望自己能够拥有一段美好的爱情,可是,这个世界上又有多少人能真正拥有一段真正的爱情呢?每个人都拥有属于自己的幸福,却又有多少人能够真正拥有属于自己的幸福呢?这或许就是“爱情诚可贵,生命价更高”的原因吧。因为只有这样,我们才能获得真正的幸福,也只有这样,我们才能真正的享受自由。而只有拥有了属于自己的幸福,我们才能够拥有真正的自由。<eot>4、对于客户所说的问题,可以用自己的方法给客户解释,这将是一种非常好的交流方式。5、客户还可以向
--------------------
CPU times: user 22.9 s, sys: 3.25 s, total: 26.1 s
Wall time: 18.4 s


In [16]:
%%time
q = '''关于亲情，达斯·维达曾经说过：“'''
ret = sample(tokenizer, gpt, q, 3, 200)
for x in ret:
    print(x)
    print('-' * 20)

关于亲情,达斯·维达曾经说过:“我的幸福感取决于我有多少时间在陪着你。”对于工作,达斯·维达也说过:“在工作中,我的幸福感取决于我在工作中的态度。”对于爱情,达斯·维达曾经说过:“我一生中只有两次爱上过男人,其中一次就是和你。”对于亲情,达斯·维达说过:“我在与我的父母一起生活的时候,我才了解到什么是真正的亲情。”对于爱情,达斯·维达说过:“我和我的爱人生活在一起的时候,我才真正了解什么是真正的爱情。”对于亲情,达斯·维达说过:“我的孩子,我的朋友,我的一切都和他们在一起。”对于爱情,达斯·维达说过:“我不希望自己成为生活的主宰,我只希望
--------------------
关于亲情,达斯·维达曾经说过:“父亲只是一个遥远的名词,对于我来说,亲情就像是阳光一样,我希望自己能够像父亲一样照顾和关心别人。”亲情,每个人都有,然而却很少有人能够很好的表达出来。亲情是最难以表达的一种情感,因为这种情感一旦表达出来,它便很难被自己的父母和亲人所理解和体会。对于孩子来说,他们需要的是父母的关心,但是却不能很好的表达出来。因为对于孩子来说,父母不一定是最了解自己的人,但是父母一定是最能够包容自己的人。因此,孩子的亲情表达其实是最为困难的,因为孩子难以想象父母对自己的爱和关心,在这种情况下,父母应该怎么办呢?一、倾听。当孩子诉说起他的心情时,父母不要只是一个倾听者,而是应该耐心的倾听孩子的诉说,用自己的
--------------------
关于亲情,达斯·维达曾经说过:“爱,是一种非常微妙的东西,它不像物质那样直接与人发生关系,但它却能以一种难以形容的方式深深地影响一个人的心灵。”父母是孩子的第一任老师,家长的一言一行对孩子起着潜移默化的作用。父母的健康心理对孩子的成长起着至关重要的作用。因此,家长要加强自身修养,培养孩子良好的性格和高尚的道德情操,努力使自己成为一个身心健康、富有爱心、有责任感的人。为此,家长在日常生活中应注意以下几点:1、培养孩子高尚的道德情操古人云:“百善孝为先”。父母的一言一行对孩子起着潜移默化的作用。父母要培养孩子高尚的道德情操,做到爱父母、敬父母、爱他人、爱祖国、爱人民、爱劳动、爱科学、爱社会主义,在
--------------------
CPU times: user 22.3 s, sys: 3.11 s, total: 25.4 s

In [17]:
%%time
q = '''菜谱：
鱼香肉丝的做法
'''
ret = sample(tokenizer, gpt, q, 3, 200)
for x in ret:
    print(x)
    print('-' * 20)

菜谱:
鱼香肉丝的做法
做法:1.将肉丝用生抽、料酒、淀粉、姜丝、白糖、盐拌匀腌制片刻;
2.青红椒洗净切丝;
3.将炒锅加热,倒入花生油,将肉丝、青红椒丝和红椒丝过油后盛出;
4.留底油,爆香姜丝,将肉丝倒入煸炒;
5.煸炒至肉丝变色,放入青红椒丝和红椒丝,加入盐、糖、鸡精,继续翻炒;
6.出锅前放入香菜拌匀即可。<eot>
鱼香肉丝做法大全<eot>做法:1.将肉丝用生抽、料酒、淀粉、姜丝、白糖、盐拌匀腌制片刻;
2.青红椒洗净切丝;
3.将炒锅
--------------------
菜谱:
鱼香肉丝的做法
1.将猪里脊肉切成4厘米长、0.3厘米粗的丝,用少许盐、料酒、淀粉抓匀,腌制10分钟。
2.把泡辣椒切成小段,姜切丝,蒜瓣拍碎,葱切花。
3.炒锅置火上,放入花生油烧热,先下入姜丝、蒜瓣爆香,再下肉丝迅速炒散,肉丝变色后加入泡辣椒、白糖、醋、酱油、料酒,待肉丝炒至八成熟时,加入葱段、泡辣椒、炒出的辣椒油、醋,炒匀后,用水淀粉勾芡,即可出锅。
菜谱:
红烧肉的做法
1.猪肉洗净,切成约2厘米见方的块;
2.将锅烧热,倒入比炒菜略多的油,烧至6成热时
--------------------
菜谱:
鱼香肉丝的做法
1.锅内油烧热,下入姜丝和葱段爆香。
2.将鱼香肉丝倒入锅中炒制,炒至七成熟时,加入泡姜、泡辣椒和泡萝卜片。
3.炒至水分收干时,加入料酒和盐,炒匀后即可出锅。
小贴士:
炒鱼香肉丝的时候,加入泡姜、泡辣椒和泡萝卜片的时候,一定要多加些水,炒制的时候会更香。
鱼香肉丝怎么做好吃
1.鱼香肉丝的做法很简单,主要是把鱼香肉丝和泡萝卜一起炒制,这样口感会更香醇,而且很好的保留了鱼香肉丝的味道。
2.鱼香肉丝是川菜中很有特色的菜肴,也是做法中的关键。鱼
--------------------
CPU times: user 21.8 s, sys: 3.04 s, total: 24.9 s
Wall time: 18 s


In [18]:
%%time
q = '''他两个搀着手，说说笑笑，转回见了唐僧。唐僧道：“可曾捉得妖怪？”行者道：“那妖怪不奈战，败回钻入水去也。”三藏道：“徒弟，这怪久住于此，他知道浅深。似这般无边的弱水，又没了舟楫，须是得个知水性的，引领引领才好哩。”行者道：“正是这等说。常言道，近朱者赤，近墨者黑。那怪在此，断知水性。我们如今拿住他，且不要打杀，只教他送师父过河，再做理会。”八戒道：“哥哥不必迟疑，让你先去拿他，等老猪看守师父。”行者笑道：“贤弟呀，这桩儿我不敢说嘴。水里勾当，老孙不大十分熟。若是空走，还要捻诀，又念念避水咒，方才走得。不然，就要变化做什么鱼虾蟹鳖之类，我才去得。若论赌手段，凭你在高山云里，干什么蹊跷异样事儿，老孙都会，只是水里的买卖，有些儿榔杭。”八戒道：“老猪当年总督天河，掌管了八万水兵大众，倒学得知些水性，却只怕那水里有什么眷族老小，七窝八代的都来，我就弄他不过，一时不被他捞去耶？”行者道：“你若到他水中与他交战，却不要恋战，许败不许胜，把他引将出来，等老孙下手助你。”八戒道：“言得是，我去耶。”说声去，就剥了青锦直裰，脱了鞋，双手舞钯，分开水路，使出那当年的旧手段，跃浪翻波，撞将进去，径至水底之下，往前正走。'''
ret = sample(tokenizer, gpt, q, 3, 200)
print(q)
print('-' * 20)
print('-' * 20)
for x in ret:
    print(x[len(q):])
    print('-' * 20)

他两个搀着手，说说笑笑，转回见了唐僧。唐僧道：“可曾捉得妖怪？”行者道：“那妖怪不奈战，败回钻入水去也。”三藏道：“徒弟，这怪久住于此，他知道浅深。似这般无边的弱水，又没了舟楫，须是得个知水性的，引领引领才好哩。”行者道：“正是这等说。常言道，近朱者赤，近墨者黑。那怪在此，断知水性。我们如今拿住他，且不要打杀，只教他送师父过河，再做理会。”八戒道：“哥哥不必迟疑，让你先去拿他，等老猪看守师父。”行者笑道：“贤弟呀，这桩儿我不敢说嘴。水里勾当，老孙不大十分熟。若是空走，还要捻诀，又念念避水咒，方才走得。不然，就要变化做什么鱼虾蟹鳖之类，我才去得。若论赌手段，凭你在高山云里，干什么蹊跷异样事儿，老孙都会，只是水里的买卖，有些儿榔杭。”八戒道：“老猪当年总督天河，掌管了八万水兵大众，倒学得知些水性，却只怕那水里有什么眷族老小，七窝八代的都来，我就弄他不过，一时不被他捞去耶？”行者道：“你若到他水中与他交战，却不要恋战，许败不许胜，把他引将出来，等老孙下手助你。”八戒道：“言得是，我去耶。”说声去，就剥了青锦直裰，脱了鞋，双手舞钯，分开水路，使出那当年的旧手段，跃浪翻波，撞将进去，径至水底之下，往前正走。
--------------------
--------------------
那怪水底下,但见:<eot>新公司产品的特点:●全自动、半自动化灌装机,包装速度快,可实现全自动的包装过程,在灌装时,只需将要灌装的容器从贮液桶中倒入,灌装头在瞬间就能将瓶口灌装好。●采用全自动的灌装设备,操作非常简单,即使是新手也能方便的操作。●采用全封闭式的工作环境,灌装时不污染环境,灌装头采用自动定位方式,灌装时,自动送料,自动出料,灌装时可直接从瓶口装入容器中,灌装完后,自动从贮液桶中取出容器,由人工取出,不需人工搬运。●该机采用进口密封件,灌装时密封性好,灌装速度快,瓶口不易泄漏。●全封闭的工作环境,灌装时不污染环境,灌装头采用
--------------------
那怪正走之间,见他来了,唬得魂不附体,只得将头钻入冰冷的水里,把钯住钯,使双拳乱打。八戒也跳入水里去战,那怪却也使起法宝,将钯来迎。两个又斗在水里,八戒使起法来,专挑那怪的头打。那怪又使起那旧手段,举钯来顶。八戒使个没遮拦招风的法宝,把他上下一齐遮拦,使钯如狂风骤雨般的乱打。那怪又使起那新手段,将钯来搭住八戒的手

In [19]:
%%time
q = '''小郡主格的一笑，掀被下床，笑道：“我穴道早解开了，等了你好久，你怎么到这时候才回来？”
韦小宝奇道：“谁给你解开穴道的？”
小郡主道：“给点了穴道，过得六七个时辰，不用解也自然通了。我扶你上床，我可得走了。”
韦小宝大急，叫道：“不行，不行。你脸上伤痕没好。须得再给你搽药，才好得全。”
小郡主嘻嘻一笑，说道：“你这人真坏，说话老骗人。你几时在我脸上刻花了？倒害得我担心了半天。”
韦小宝问道：“你怎么知道？”
小郡主道：“我早下床来照过镜子，脸上什么也没有。”
'''
ret = sample(tokenizer, gpt, q, 3, 200)
print(q)
print('-' * 20)
print('-' * 20)
for x in ret:
    print(x[len(q):])
    print('-' * 20)

小郡主格的一笑，掀被下床，笑道：“我穴道早解开了，等了你好久，你怎么到这时候才回来？”
韦小宝奇道：“谁给你解开穴道的？”
小郡主道：“给点了穴道，过得六七个时辰，不用解也自然通了。我扶你上床，我可得走了。”
韦小宝大急，叫道：“不行，不行。你脸上伤痕没好。须得再给你搽药，才好得全。”
小郡主嘻嘻一笑，说道：“你这人真坏，说话老骗人。你几时在我脸上刻花了？倒害得我担心了半天。”
韦小宝问道：“你怎么知道？”
小郡主道：“我早下床来照过镜子，脸上什么也没有。”

--------------------
--------------------
韦小宝道:“你还不知道我的厉害,知道了也不相信。”
小郡主道:“你不是坏人,就是在骗我。”
韦小宝道:“我说的话都是真话,你不信就算了。”
小郡主笑道:“我不跟你说正经话。你说过要娶我的,可不是骗我。”
韦小宝道:“不错,我说过要娶你的。”
小郡主道:“你要娶我,那是要赖帐的。”
韦小宝道:“我不跟你赌钱。你若肯嫁给我,我就跟你睡。”
小郡主道:“你说过我不能嫁你的。”
韦小宝道:“你若答应嫁给我,就不能反悔。”
小郡主道
--------------------
韦小宝道:“我可什么也没瞧见。”
小郡主道:“我又没戴面纱,你怎么知道?”
韦小宝叫道:“我......我......”
小郡主笑道:“你急什么?我脸上的伤痕,可也是骗你的。”
韦小宝道:“不是我说的,是我自己看出来的。”
小郡主道:“你若真是看出来的,为什么不说?”
韦小宝道:“我是大笨蛋,怎知你骗我?”
小郡主笑道:“你是大笨蛋,别人可还不知道呢。”
韦小宝道:“你脸上的伤,是给那个叫做张无忌的小子打的?”
小郡主道:“嗯,是给他打的。”
--------------------
韦小宝道:“你别笑,我不跟你说笑的。你这脸真得很好,跟我第一次见到你的时候,真是一模一样。”
小郡主道:“我没有骗你,你脸上刻的花是我刻上去的。”
韦小宝大奇,心想:“你这小姑娘不知打哪儿学的法子,居然连自己脸上的伤疤也会骗人。”说道:“是吗?你怎么知道?”
小郡主道:“我的眼睛是假的,你不认得我,怎知我是假的?我脸上伤疤的颜色,跟你脸上一样。”
韦小宝道:“那你怎么不来服侍我?”
小郡主道:“你不喜欢我,我怎会来服侍你?”
韦小宝喜道:“真的?”
------------------

In [20]:
%%time
q = '''上联：天地一统，脚踩万千山河
下联：'''
ret = sample(tokenizer, gpt, q, 3, 20)
print(q)
print('-' * 20)
print('-' * 20)
for x in ret:
    print(x[len(q):])
    print('-' * 20)

上联：天地一统，脚踩万千山河
下联：
--------------------
--------------------
万物归一,脚踩万千世界
上联:天地一统,脚踩万千山河
--------------------
九州方圆,俯瞰大地风云
横批:大美中国<eot>上联:天地一统
--------------------
八方来朝,口接千载明月
上联:天地一统,脚踩万千山河
--------------------
CPU times: user 2.42 s, sys: 346 ms, total: 2.77 s
Wall time: 1.88 s


In [21]:
%%time
q = '''算术题
1+3=4
3+2=5
1+7='''
ret = sample(tokenizer, gpt, q, 3, 5)
print(q)
print('-' * 20)
print('-' * 20)
for x in ret:
    print(x[len(q):])
    print('-' * 20)

算术题
1+3=4
3+2=5
1+7=
--------------------
--------------------
9
4+5
--------------------
11
1+2
--------------------
9
2+2
--------------------
CPU times: user 841 ms, sys: 103 ms, total: 944 ms
Wall time: 609 ms


In [22]:
%%time
q = '''这个人的开放程度是高还是低？
问：很有创意
答：高
问：不喜欢改变
答：低
问：乐于尝试新事物
答：高
问：专注于应对新挑战
答：高
问：不喜欢新事物
答：低
问：乐于思考抽象概念
答：高
问：抵制新想法
答：低
问：想象力不丰富
答：低
问：不喜欢抽象或理论性的概念
答：低
问：喜欢智力挑战
答：高
问：听故事很开心
答：高
问：不会加入少数意见
答：低
问：永远只支持leader
答：低
问：会考虑替代领导者
答：高
问：真的很喜欢哲学
答：高
问：宁愿避免遇到他的敌人
答：低
问：喜欢与人交流
答：'''
ret = sample(tokenizer, gpt, q, 3, 1)
print(q)
print('-' * 20)
print('-' * 20)
for x in ret:
    print(x[len(q):])
    print('-' * 20)

这个人的开放程度是高还是低？
问：很有创意
答：高
问：不喜欢改变
答：低
问：乐于尝试新事物
答：高
问：专注于应对新挑战
答：高
问：不喜欢新事物
答：低
问：乐于思考抽象概念
答：高
问：抵制新想法
答：低
问：想象力不丰富
答：低
问：不喜欢抽象或理论性的概念
答：低
问：喜欢智力挑战
答：高
问：听故事很开心
答：高
问：不会加入少数意见
答：低
问：永远只支持leader
答：低
问：会考虑替代领导者
答：高
问：真的很喜欢哲学
答：高
问：宁愿避免遇到他的敌人
答：低
问：喜欢与人交流
答：
--------------------
--------------------
高
--------------------
高
--------------------
高
--------------------
CPU times: user 299 ms, sys: 88.9 ms, total: 388 ms
Wall time: 427 ms


In [23]:
%%time
q = '''这个人的开放程度是高还是低？
问：很有创意
答：高
问：不喜欢改变
答：低
问：乐于尝试新事物
答：高
问：专注于应对新挑战
答：高
问：不喜欢新事物
答：低
问：乐于思考抽象概念
答：高
问：抵制新想法
答：低
问：想象力不丰富
答：低
问：不喜欢抽象或理论性的概念
答：低
问：喜欢智力挑战
答：高
问：听故事很开心
答：高
问：不会加入少数意见
答：低
问：永远只支持leader
答：低
问：会考虑替代领导者
答：高
问：真的很喜欢哲学
答：高
问：宁愿避免遇到他的敌人
答：低
问：讨厌与人交流
答：'''
ret = sample(tokenizer, gpt, q, 3, 1)
print(q)
print('-' * 20)
print('-' * 20)
for x in ret:
    print(x[len(q):])
    print('-' * 20)

这个人的开放程度是高还是低？
问：很有创意
答：高
问：不喜欢改变
答：低
问：乐于尝试新事物
答：高
问：专注于应对新挑战
答：高
问：不喜欢新事物
答：低
问：乐于思考抽象概念
答：高
问：抵制新想法
答：低
问：想象力不丰富
答：低
问：不喜欢抽象或理论性的概念
答：低
问：喜欢智力挑战
答：高
问：听故事很开心
答：高
问：不会加入少数意见
答：低
问：永远只支持leader
答：低
问：会考虑替代领导者
答：高
问：真的很喜欢哲学
答：高
问：宁愿避免遇到他的敌人
答：低
问：讨厌与人交流
答：
--------------------
--------------------
高
--------------------
高
--------------------
高
--------------------
CPU times: user 284 ms, sys: 47.6 ms, total: 332 ms
Wall time: 404 ms
